In [2]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
from sklearn.datasets import make_blobs
import mglearn
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV


# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#style.use('ggplot') or plt.style.use('ggplot')

# Preprocesado y modelado
# ==============================================================================
from sklearn.ensemble import IsolationForest
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')


import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
#Random global
seed = 12345
rng = np.random.default_rng(seed)  # can be called without a seed
rng.random()
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
df = pd.read_csv (r"C:\Users\White\Desktop\tfg\generados\RFECved.csv")
df2 = pd.read_csv (r"C:\Users\White\Desktop\tfg\generados\scaled.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334380 entries, 0 to 334379
Data columns (total 31 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Src Port            334380 non-null  float64
 1   Dst Port            334380 non-null  float64
 2   Tot Fwd Pkts        334380 non-null  float64
 3   Fwd Pkt Len Min     334380 non-null  float64
 4   Fwd Pkt Len Mean    334380 non-null  float64
 5   Bwd Pkt Len Max     334380 non-null  float64
 6   Bwd Pkt Len Min     334380 non-null  float64
 7   Flow Byts/s         334380 non-null  float64
 8   Flow IAT Mean       334380 non-null  float64
 9   Flow IAT Min        334380 non-null  float64
 10  Fwd IAT Tot         334380 non-null  float64
 11  Fwd IAT Mean        334380 non-null  float64
 12  Fwd IAT Max         334380 non-null  float64
 13  Bwd IAT Mean        334380 non-null  float64
 14  Fwd Header Len      334380 non-null  float64
 15  Bwd Header Len      334380 non-nul

In [4]:
#Loading the dataset
X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")


# GaussianNB as a baseline

In [15]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
import sklearn.metrics as metrics

X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")

#because only var_smoothing can be 'tuned'
#do a cross validation on different var_smoothing values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=42)


accuracy=[]
precision=[]
recall=[]
f1_score=[]
confusion_matrix=[]
time=[]
best_params=[]

#baseline parameters

p_grid = { 
    'var_smoothing': [1e-8, 1e-7, 1e-6, 1e-5, 1e-4]
}

scoring = {"Accuracy": metrics.make_scorer(metrics.accuracy_score)}

#will contain the cv results
results = []

outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X.iloc[train_index,:], X.iloc[test_index,:] 
    y_train,y_test= y.iloc[train_index], y.iloc[test_index]
        
    grid = GridSearchCV(GaussianNB(),return_train_score=True,scoring="accuracy", refit=True, param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=3)
    grid.fit(X_train, y_train)
    
    
    #print("Best cross validation accuracy: {:.12f}".format(grid.best_score_))
    #print("Test set score: {:.12f}".format(grid.score(X_test,y_test)))
    #print("Best parameters: {}".format(grid.best_params_))
    #best_params.append(grid.best_params_)
    
    #Mas parametros a conseguir
    
    #y_pred=grid.predict(X_test)
    #accuracy.append(metrics.accuracy_score(y_test,y_pred))
    #precision.append(metrics.precision_score(y_test,y_pred))
    #recall.append(metrics.recall_score(y_test,y_pred))
    #f1_score.append(metrics.f1_score(y_test,y_pred))
    #confusion_matrix.append(metrics.confusion_matrix(y_test,y_pred))
    #parametros=grid.cv_results_['params']

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits


In [16]:
pd.DataFrame(grid.cv_results_).to_csv(r"C:\Users\White\Desktop\tfg\generados\GaussianGridSearchCV.csv", index = None, header=True) 

# RandomForest

In [17]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics

X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=42)


# define the model with default hyperparameters
model=RandomForestClassifier(random_state=42)

# define the grid of values to search
p_grid = { 
    'n_estimators': [100, 200],
    'max_features': ['auto', 'sqrt'],
    'max_depth' : [4,5],
    'criterion' :['gini', 'entropy']
}

outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X.iloc[train_index,:], X.iloc[test_index,:] 
    y_train,y_test= y.iloc[train_index], y.iloc[test_index]
        
    grid = GridSearchCV(model,return_train_score=True,scoring="accuracy",param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=4)
    grid.fit(X_train, y_train)
  

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [18]:
pd.DataFrame(grid.cv_results_).to_csv(r"C:\Users\White\Desktop\tfg\generados\RForestGridSearchCV.csv", index = None, header=True) 

# Adaboost

In [19]:
import pandas as pd
from sklearn.model_selection import cross_val_score
import sklearn.metrics as metrics

X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=42)


from sklearn.ensemble import AdaBoostClassifier
# define the model with default hyperparameters
model = AdaBoostClassifier()
# define the grid of values to search
p_grid = dict()
p_grid['n_estimators'] = [10,50,100]
p_grid['learning_rate'] = [0.1,0.001,0.0001]
scoring = {"AUC": "roc_auc", "Accuracy": metrics.make_scorer(metrics.accuracy_score), "Precision":metrics.make_scorer(metrics.precision_score)}

#will contain the cv results
results = []

outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X.iloc[train_index,:], X.iloc[test_index,:] 
    y_train,y_test= y.iloc[train_index], y.iloc[test_index]
        
    grid = GridSearchCV(model,return_train_score=True,scoring="accuracy",param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=4)
    grid.fit(X_train, y_train)
  

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits


In [22]:
pd.DataFrame(grid.cv_results_).to_csv(r"C:\Users\White\Desktop\tfg\generados\AdaboostGridSearchCV.csv", index = None, header=True) 

# LinearSVC

In [23]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
import sklearn.metrics as metrics

X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=42)

# define the model with default hyperparameters
model = LinearSVC()
# define the grid of values to search
p_grid = {'C':[0.1,10,100]}

#will contain the cv results
results = []

outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X.iloc[train_index,:], X.iloc[test_index,:] 
    y_train,y_test= y.iloc[train_index], y.iloc[test_index]
        
    grid = GridSearchCV(model,return_train_score=True,scoring="accuracy",param_grid=p_grid, n_jobs=2, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=4)
    grid.fit(X_train, y_train)
  

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits


In [24]:
pd.DataFrame(grid.cv_results_).to_csv(r"C:\Users\White\Desktop\tfg\generados\LinearSVCGridSearchCV.csv", index = None, header=True) 

# Autoencoder

In [6]:
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam


X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=42)

#Autoencoder
# Function to create model, required for KerasClassifier
def create_model(learning_rate=0.01):
    #Enc Dimension 
    encoding_dim=100
    #Input shape
    input_dim = Input(shape=(31,))
    #Encoding Layer
    encoded = Dense(encoding_dim, activation='relu')(input_dim)
    #Decoding Layer
    decoded = Dense(1, activation='sigmoid')(encoded)

    #Model AE
    autoencoder = Model(input_dim, decoded)
    #Model Encoder 
    encoder = Model(input_dim, encoded)
    #Encoding
    encoded_input = Input(shape=(encoding_dim,))
    #Decoding 
    decoder_layer = autoencoder.layers[-1]
    #Model Decoder 
    decoder = Model(encoded_input, decoder_layer(encoded_input))

    #optimizer=tf.keras.optimizers.Adadelta(lr=0.1, rho=0.95, epsilon=None, decay=0.0)
    autoencoder.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    model = autoencoder

    # Compile model
    return model




model = KerasClassifier(build_fn = create_model)



# define the grid search parameters
#batch_size = [10, 20, 40]
#epochs = [10, 30]
#p_grid = dict(batch_size=batch_size, epochs=epochs)
#batch_size = [10,128]
#epochs = [10,50]
learning_rate = [ 0.1, 0.2, 0.3]
momentum = [0.8, 0.9]


# define the grid search parameters
batch_size = [100, 200]
epochs = [10, 30]
#dropout = [0, 0.1, 0.2]
p_grid = dict(batch_size=batch_size, epochs=epochs, learning_rate=learning_rate)

outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X.iloc[train_index,:], X.iloc[test_index,:] 
    y_train,y_test= y.iloc[train_index], y.iloc[test_index]
        
    grid = GridSearchCV(model,return_train_score=True,scoring="accuracy",param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=4)
    grid.fit(X_train, y_train)


Fitting 5 folds for each of 12 candidates, totalling 60 fits


exception calling callback for <Future at 0x13280c4bb20 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "C:\Users\White\anaconda3\lib\site-packages\joblib\externals\loky\_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "C:\Users\White\anaconda3\lib\site-packages\joblib\parallel.py", line 359, in __call__
    self.parallel.dispatch_next()
  File "C:\Users\White\anaconda3\lib\site-packages\joblib\parallel.py", line 794, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "C:\Users\White\anaconda3\lib\site-packages\joblib\parallel.py", line 861, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\White\anaconda3\lib\site-packages\joblib\parallel.py", line 779, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "C:\Users\White\anaconda3\lib\site-packages\joblib\_parallel_backends.py", line 531, in apply_async
    future = self._workers.submit(SafeFunction(func))


OSError: [Errno 22] Invalid argument

In [4]:
pd.DataFrame(grid.cv_results_).to_csv(r"C:\Users\White\Desktop\tfg\generados\AutoencoderGridSearchCV.csv", index = None, header=True) 

# TensorBoard tryout

In [18]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp


HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
def train_test_model(hparams):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
  ])
  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'],
  )

  model.fit(X_train, y_train, epochs=1) # Run with 1 epoch to speed things up for demo purposes
  _, accuracy = model.evaluate(X_test, y_test)
  return accuracy

session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam'}
2090/2090 [==============================] - 2s 719us/step - loss: 0.0011 - accuracy: 0.9998
--- Starting trial: run-1
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'sgd'}
2090/2090 [==============================] - 2s 710us/step - loss: 0.0078 - accuracy: 0.9981
--- Starting trial: run-2
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'adam'}
2090/2090 [==============================] - 2s 726us/step - loss: 0.0013 - accuracy: 0.9991
--- Starting trial: run-3
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'sgd'}
2090/2090 [==============================] - 2s 728us/step - loss: 0.0077 - accuracy: 0.9980
--- Starting trial: run-4
{'num_units': 32, 'dropout': 0.1, 'optimizer': 'adam'}
2090/2090 [==============================] - 2s 723us/step - loss: 5.9942e-04 - accuracy: 0.9998
--- Starting trial: run-5
{'num_units'

In [19]:
%tensorboard --logdir logs/hparam_tuning



# RNN

In [56]:
X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=42)

#Numpy has far better performance on any operations 
#X_train=X_train.to_numpy()
#X_test=X_test.to_numpy()
#y_train=y_train.to_numpy()
#y_test=y_test.to_numpy()


y_train.shape

(250785,)

In [70]:
# Part 2 - Building the RNN

# Importing the Keras libraries and packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM, Dropout, Input
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers



hidden_nodes = int(2/3 * (X_train.shape[1] + 1))
#print(f"The number of hidden nodes is {hidden_nodes}.")

# Function to create model, required for KerasClassifier
# Initialising the RNN
def create_model():
    #Initializing the RNN
    model = Sequential()
    #Adding the first LSTM layer and dropout 
    model.add(LSTM(units=21, return_sequences=False, input_shape=(31, 1)))
    model.add(Dropout(0.2))
    #Adding a second LSTM Layer and dropout
    #model.add(LSTM(units=20, return_sequences=False))
    #model.add(Dropout(0.2))
    #Adding the output layer
    model.add(Dense(units=1,activation='sigmoid'))
    #Output is two classes with only one present at a time. 
    #Softmax allows the model to interpret the outputs as probabilities

    #Compiling the model without training
    #Loss function is chosen together with activation. Softmax points us to binary crossentropy
    #since we are faced with a binary classification problem
    from tensorflow.keras.optimizers import SGD
    opt = SGD(lr=0.01)

    model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=['acc'])
    return model

# define the grid search parameters
#optimizer= ["RMSprop", "adam"]
epochs = [10]
#dropout = [0, 0.1, 0.2]
p_grid = dict(batch_size=batch_size, epochs=epochs)

model = KerasClassifier(build_fn = create_model)
model.fit(X_train, y_train, batch_size=1000, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
251/251 [==============================] - 15s 53ms/step - loss: 0.3073 - acc: 0.8619 - val_loss: 0.2014 - val_acc: 0.8959
Epoch 2/10
251/251 [==============================] - 13s 51ms/step - loss: 0.1449 - acc: 0.9229 - val_loss: 0.0604 - val_acc: 0.9821
Epoch 3/10
251/251 [==============================] - 13s 52ms/step - loss: 0.0409 - acc: 0.9894 - val_loss: 0.0453 - val_acc: 0.9890
Epoch 4/10
251/251 [==============================] - 13s 51ms/step - loss: 0.0263 - acc: 0.9929 - val_loss: 0.0211 - val_acc: 0.9935
Epoch 5/10
251/251 [==============================] - 13s 52ms/step - loss: 0.0180 - acc: 0.9953 - val_loss: 0.0156 - val_acc: 0.9965
Epoch 6/10
251/251 [==============================] - 13s 52ms/step - loss: 0.0137 - acc: 0.9966 - val_loss: 0.0147 - val_acc: 0.9964
Epoch 7/10
251/251 [==============================] - 13s 51ms/step - loss: 0.0118 - acc: 0.9973 - val_loss: 0.0110 - val_acc: 0.9975
Epoch 8/10
251/251 [==============================] - 13s 51ms

In [71]:
# define the grid search parameters
batch_size = [1000,5000]
#optimizer= ["RMSprop", "adam"]
epochs = [10, 20]
#dropout = [0, 0.1, 0.2]
p_grid = dict(batch_size=batch_size, epochs=epochs)


outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X.iloc[train_index,:], X.iloc[test_index,:] 
    y_train,y_test= y.iloc[train_index], y.iloc[test_index]
        
    grid = GridSearchCV(model,return_train_score=True,scoring="accuracy",param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=4)
    grid.fit(X_train, y_train)



Fitting 5 folds for each of 4 candidates, totalling 20 fits
Epoch 1/20
536/536 [==============================] - 16s 26ms/step - loss: 0.2133 - acc: 0.9037
Epoch 2/20
536/536 [==============================] - 14s 26ms/step - loss: 0.0412 - acc: 0.9892
Epoch 3/20
536/536 [==============================] - 14s 26ms/step - loss: 0.0206 - acc: 0.9948
Epoch 4/20
536/536 [==============================] - 14s 26ms/step - loss: 0.0141 - acc: 0.9969
Epoch 5/20
536/536 [==============================] - 14s 26ms/step - loss: 0.0118 - acc: 0.9975
Epoch 6/20
536/536 [==============================] - 14s 26ms/step - loss: 0.0097 - acc: 0.9980
Epoch 7/20
536/536 [==============================] - 14s 26ms/step - loss: 0.0083 - acc: 0.9983
Epoch 8/20
536/536 [==============================] - 14s 26ms/step - loss: 0.0072 - acc: 0.9985
Epoch 9/20
536/536 [==============================] - 14s 26ms/step - loss: 0.0062 - acc: 0.9987
Epoch 10/20
536/536 [==============================] - 14s 26ms/ste

536/536 [==============================] - 16s 26ms/step - loss: 0.2181 - acc: 0.9067
Epoch 2/20
536/536 [==============================] - 14s 26ms/step - loss: 0.0295 - acc: 0.9922
Epoch 3/20
536/536 [==============================] - 14s 26ms/step - loss: 0.0140 - acc: 0.9969
Epoch 4/20
536/536 [==============================] - 14s 26ms/step - loss: 0.0101 - acc: 0.9980
Epoch 5/20
536/536 [==============================] - 14s 26ms/step - loss: 0.0082 - acc: 0.9985
Epoch 6/20
536/536 [==============================] - 14s 26ms/step - loss: 0.0056 - acc: 0.9989
Epoch 7/20
536/536 [==============================] - 14s 26ms/step - loss: 0.0038 - acc: 0.9992
Epoch 8/20
536/536 [==============================] - 14s 26ms/step - loss: 0.0028 - acc: 0.9994
Epoch 9/20
536/536 [==============================] - 14s 26ms/step - loss: 0.0020 - acc: 0.9995
Epoch 10/20
536/536 [==============================] - 14s 26ms/step - loss: 0.0016 - acc: 0.9996
Epoch 11/20
536/536 [===================

In [72]:
pd.DataFrame(grid.cv_results_).to_csv(r"C:\Users\White\Desktop\tfg\generados\RNNGridSearchCV.csv", index = None, header=True) 

In [ ]:
pd.DataFrame(grid.cv_results_).to_csv(r"C:\Users\White\Desktop\tfg\generados\RNNGridSearchCV.csv", index = None, header=True) 

# Building the CNN


In [79]:
#Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers

X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=42)

X_train=X_train.to_numpy()
X_train = X_train[..., np.newaxis]
X_test=X_test.to_numpy()
X_test = X_test[..., np.newaxis]

y_train= y_train.to_numpy()
y_train = y_train[..., np.newaxis]
y_test= y_test.to_numpy()
y_test = y_test[..., np.newaxis]



# Function to create model, required for KerasClassifier
# Initialising the CNN
def create_model():
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1],X_train.shape[2] )))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.2))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



model = KerasClassifier(build_fn = create_model)
model.fit(X_train, y_train, batch_size=1000, epochs=10, validation_data=(X_test, y_test))




Epoch 1/10
251/251 [==============================] - 19s 76ms/step - loss: 0.0353 - accuracy: 0.9893 - val_loss: 0.0037 - val_accuracy: 0.9994
Epoch 2/10
251/251 [==============================] - 19s 75ms/step - loss: 0.0039 - accuracy: 0.9993 - val_loss: 0.0016 - val_accuracy: 0.9997
Epoch 3/10
251/251 [==============================] - 19s 75ms/step - loss: 0.0027 - accuracy: 0.9996 - val_loss: 0.0012 - val_accuracy: 0.9999
Epoch 4/10
251/251 [==============================] - 19s 75ms/step - loss: 0.0017 - accuracy: 0.9997 - val_loss: 7.7088e-04 - val_accuracy: 0.9999
Epoch 5/10
251/251 [==============================] - 19s 75ms/step - loss: 0.0018 - accuracy: 0.9997 - val_loss: 5.2818e-04 - val_accuracy: 0.9999
Epoch 6/10
251/251 [==============================] - 19s 75ms/step - loss: 0.0012 - accuracy: 0.9998 - val_loss: 8.1094e-04 - val_accuracy: 0.9999
Epoch 7/10
251/251 [==============================] - 19s 75ms/step - loss: 9.4191e-04 - accuracy: 0.9998 - val_loss: 2.8747

In [ ]:

# define the grid search parameters
batch_size = [32, 64]
#optimizer= ["RMSprop", "adam"]
epochs = [10, 20]
#dropout = [0, 0.1, 0.2]
p_grid = dict(batch_size=batch_size, epochs=epochs)



outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X.iloc[train_index,:], X.iloc[test_index,:] 
    y_train,y_test= y.iloc[train_index], y.iloc[test_index]
        
    grid = GridSearchCV(model,return_train_score=True,scoring="accuracy",param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=4)
    grid.fit(X_train, y_train)


In [ ]:
pd.DataFrame(grid.cv_results_).to_csv(r"C:\Users\White\Desktop\tfg\generados\CNNGridSearchCV.csv", index = None, header=True) 

In [30]:
nX_train=X_train.to_numpy()
nX_train = nX_train[..., np.newaxis]

In [32]:
nX_train

array([[[-9.13798267e-01],
        [ 2.06517507e+00],
        [-6.23681245e-02],
        ...,
        [-1.60936040e+00],
        [-6.42964049e-01],
        [ 2.12768266e+00]],

       [[ 1.74578570e+00],
        [-4.03402134e-01],
        [-6.23681245e-02],
        ...,
        [ 6.21364859e-01],
        [-6.42964049e-01],
        [-4.69994900e-01]],

       [[ 1.46934696e+00],
        [-4.92854044e-01],
        [ 6.85065613e-04],
        ...,
        [ 6.21364859e-01],
        [-6.42964049e-01],
        [-4.69994900e-01]],

       ...,

       [[-9.13798267e-01],
        [ 2.10691519e+00],
        [-6.23681245e-02],
        ...,
        [-1.60936040e+00],
        [-6.42964049e-01],
        [ 2.12768266e+00]],

       [[ 1.04284389e+00],
        [ 1.56518800e-01],
        [-6.23681245e-02],
        ...,
        [ 6.21364859e-01],
        [-6.42964049e-01],
        [-4.69994900e-01]],

       [[ 8.18404276e-01],
        [-4.92854044e-01],
        [ 6.85065613e-04],
        ...,
        